# Tutorial: Deadline-Aware Worker Pool with Queue

**Category**: Concurrency
**Difficulty**: Advanced
**Time**: 25-35 minutes

## Problem Statement

When building high-throughput data processing systems, you often need multiple concurrent workers processing tasks from a shared queue. Examples include API request handlers, batch data transformers, or distributed job processors. The challenge is coordinating multiple workers to process as much work as possible within a deadline while handling failures gracefully and shutting down cleanly.

Consider a data pipeline that needs to process 10,000 records before a nightly ETL window closes. You can't process them sequentially (too slow), but you also can't spawn 10,000 concurrent tasks (resource exhaustion). You need a fixed-size worker pool that pulls from a queue, processes items concurrently, and stops when either the queue is empty or the deadline is reached. Workers must also handle individual task failures without crashing and shut down cleanly without leaving orphaned tasks.

**Why This Matters**:
- **Throughput**: Worker pools provide controlled parallelism - process 10-100x faster than sequential while avoiding resource exhaustion
- **Reliability**: Individual worker failures don't stop the entire pool, and clean shutdown prevents data loss
- **Deadline Compliance**: Production systems have SLAs - ETL windows, report generation deadlines, batch processing time limits

**What You'll Build**:
A production-ready worker pool using lionherd-core's `Queue.with_maxsize()`, `effective_deadline()`, and structured concurrency that processes tasks with multiple concurrent workers, tracks progress across workers, handles errors gracefully, and shuts down cleanly using the sentinel pattern.

## Prerequisites

**Prior Knowledge**:
- Python async/await and task groups
- Queue-based work distribution patterns
- Exception handling in concurrent contexts
- Understanding of producer-consumer patterns

**Required Packages**:
```bash
pip install lionherd-core  # >=0.1.0
```

**Optional Reading**:
- [API Reference: Primitives](../../docs/api/libs/concurrency/primitives.md)
- [API Reference: Cancellation](../../docs/api/libs/concurrency/cancel.md)
- [Reference Notebook: Concurrency Primitives](../references/concurrency_primitives.ipynb)

In [ ]:
# Standard library
import asyncio
from dataclasses import dataclass, field
from enum import Enum
from typing import Any, Callable, Generic, TypeVar

# lionherd-core
from lionherd_core.libs.concurrency import (
    Queue,
    current_time,
    effective_deadline,
    fail_at,
    move_on_at,
    sleep,
)

T = TypeVar("T")
R = TypeVar("R")

## Solution Overview

We'll implement a worker pool pattern using these components:

1. **Queue Management**: Bounded queue with `Queue.with_maxsize()` to control memory usage
2. **Worker Pool**: Fixed number of concurrent workers pulling from shared queue
3. **Sentinel Pattern**: Special "stop" token to signal workers to shut down gracefully
4. **Deadline Awareness**: Each worker checks `effective_deadline()` before processing tasks

**Key lionherd-core Components**:
- `Queue.with_maxsize(n)`: Bounded FIFO queue with backpressure
- `effective_deadline()`: Query remaining time from ambient cancel scopes
- `move_on_at(deadline)`: Silent cancellation at absolute deadline
- `fail_at(deadline)`: Raises TimeoutError at deadline (for strict mode)

**Flow**:
```
Producer → Queue (maxsize=100) → Worker 1 → Results
                              ↘ Worker 2 → Results
                              ↘ Worker 3 → Results
                              ↘ Worker N → Results
                              
Sentinel → Workers stop gracefully
Deadline → All workers cancelled
```

**Expected Outcome**: Process tasks concurrently with controlled parallelism, respecting deadlines, handling errors per-worker, and shutting down cleanly without orphaned tasks.

In [ ]:
# Quick Start: Deadline-Aware Workers in 30 Seconds

from lionherd_core.libs.concurrency import Queue, effective_deadline, current_time, move_on_at, sleep
import asyncio

SENTINEL = object()  # Shutdown signal

async def deadline_aware_worker(worker_id: int, queue: Queue, results: list):
    """Worker that checks deadline before each task."""
    while True:
        item = await queue.get()
        if item is SENTINEL:
            await queue.put(SENTINEL)
            break
        
        # Check remaining time
        deadline = effective_deadline()
        if deadline and (deadline - current_time()) < 0.1:
            print(f"Worker {worker_id}: Skipping {item} (deadline near)")
            continue
        
        await sleep(0.05)
        results.append(f"W{worker_id}:{item}")

# Try it with 1 second deadline:
queue = Queue.with_maxsize(20)
results = []

async with asyncio.TaskGroup() as tg:
    deadline = current_time() + 1.0
    
    with move_on_at(deadline):
        for i in range(3):
            tg.create_task(deadline_aware_worker(i, queue, results))
        
        for i in range(50):
            await queue.put(f"task-{i}")
        await queue.put(SENTINEL)

print(f"Processed {len(results)}/50 tasks within deadline")

# 👇 Now read below for production worker pools with full error handling

### Step 1: Define Task and Sentinel Pattern

The sentinel pattern uses a special marker value to signal "no more work". When a worker pulls the sentinel from the queue, it knows to exit its processing loop. This enables graceful shutdown without cancellation.

**Why Sentinel**: Cancellation is abrupt and can leave tasks partially processed. Sentinels let workers finish their current task and exit cleanly.

In [ ]:
class _Sentinel:
    """Sentinel marker for queue shutdown."""
    
    def __repr__(self) -> str:
        return "SENTINEL"


SENTINEL = _Sentinel()


@dataclass
class WorkItem(Generic[T]):
    """Item to process in worker pool."""
    
    id: str
    data: T
    priority: int = 0
    metadata: dict[str, Any] = field(default_factory=dict)
    
    def __repr__(self) -> str:
        return f"WorkItem(id={self.id!r}, priority={self.priority})"


class WorkStatus(Enum):
    """Processing outcome."""
    
    SUCCESS = "success"
    FAILED = "failed"
    TIMEOUT = "timeout"  # Worker ran out of time


@dataclass
class WorkResult(Generic[R]):
    """Result from processing a work item."""
    
    item_id: str
    worker_id: int
    status: WorkStatus
    result: R | None = None
    error: Exception | None = None
    duration: float = 0.0
    
    def __repr__(self) -> str:
        return f"WorkResult(id={self.item_id!r}, worker={self.worker_id}, status={self.status.value}, duration={self.duration:.3f}s)"


# Example usage
item = WorkItem(id="item-001", data={"user_id": 123}, priority=1)
result = WorkResult(item_id="item-001", worker_id=2, status=WorkStatus.SUCCESS, duration=0.15)

print(f"Work Item: {item}")
print(f"Result: {result}")
print(f"Sentinel: {SENTINEL}")

**Notes**:
- **Sentinel Pattern**: `SENTINEL` is a singleton object. Workers check `item is SENTINEL` (identity check).
- **Worker ID**: Each result tracks which worker processed it, enabling per-worker performance analysis.
- **Generic Types**: `WorkItem[T]` and `WorkResult[R]` support any data type and result type.
- **Timeout Status**: Distinguishes between task failures (bad data, API errors) and deadline exhaustion.

### Step 2: Implement Single Worker Logic

Each worker runs an infinite loop: pull item from queue, check deadline, process item, record result. The loop exits on sentinel or when deadline is reached.

**Why Check Deadline**: Prevents workers from starting tasks they can't finish. If only 5 seconds remain and the task typically takes 30 seconds, skip it.

In [ ]:
async def worker(
    worker_id: int,
    queue: Queue[WorkItem[T] | _Sentinel],
    processor: Callable[[T], R],
    results: list[WorkResult[R]],
    min_time_remaining: float = 0.1,
) -> None:
    """Single worker that processes items from queue.
    
    Args:
        worker_id: Unique worker identifier
        queue: Shared work queue
        processor: Async function to process work item data
        results: Shared list to append results
        min_time_remaining: Skip task if less than this time left
    """
    while True:
        # Pull next item from queue
        item = await queue.get()
        
        # Check for sentinel (graceful shutdown)
        if item is SENTINEL:
            # Put sentinel back for other workers
            await queue.put(SENTINEL)
            break
        
        # Check remaining time
        deadline = effective_deadline()
        if deadline is not None:
            remaining = deadline - current_time()
            if remaining < min_time_remaining:
                # Not enough time, mark as timeout
                results.append(WorkResult(
                    item_id=item.id,
                    worker_id=worker_id,
                    status=WorkStatus.TIMEOUT,
                ))
                # Put item back in queue for potential retry
                await queue.put(item)
                continue
        
        # Process the item
        start = current_time()
        try:
            result = await processor(item.data)
            duration = current_time() - start
            results.append(WorkResult(
                item_id=item.id,
                worker_id=worker_id,
                status=WorkStatus.SUCCESS,
                result=result,
                duration=duration,
            ))
        except Exception as e:
            duration = current_time() - start
            results.append(WorkResult(
                item_id=item.id,
                worker_id=worker_id,
                status=WorkStatus.FAILED,
                error=e,
                duration=duration,
            ))


# Test single worker
async def simple_processor(data: dict) -> str:
    """Simulate processing."""
    await sleep(0.1)
    return f"Processed user {data['user_id']}"


# Create queue with 5 items
test_queue = Queue.with_maxsize(10)
for i in range(5):
    await test_queue.put(WorkItem(id=f"item-{i}", data={"user_id": i}))
await test_queue.put(SENTINEL)

# Run single worker
test_results = []
await worker(worker_id=1, queue=test_queue, processor=simple_processor, results=test_results)

print(f"Processed {len(test_results)} items:")
for r in test_results:
    print(f"  {r}")

**Notes**:
- **Sentinel Propagation**: When a worker sees `SENTINEL`, it puts it back in the queue so other workers can also see it and exit.
- **Time-Aware Processing**: Workers check `effective_deadline()` before each task, not just at the start. This respects ambient cancel scopes.
- **Item Re-queuing**: When a worker times out, it puts the item back in the queue. In production, you might want a separate "timeout" queue or retry limit.
- **Shared Results**: All workers append to the same `results` list. This is safe because Python's GIL ensures list.append() is atomic.

### Step 3: Build Worker Pool Coordinator

The coordinator spawns multiple workers, feeds the queue, and manages shutdown. It uses structured concurrency to ensure all workers are cleaned up properly.

**Why Structured Concurrency**: If the coordinator is cancelled (deadline reached), all workers are automatically cancelled. No orphaned tasks or resource leaks.

In [ ]:
async def run_worker_pool(
    items: list[WorkItem[T]],
    processor: Callable[[T], R],
    num_workers: int = 4,
    queue_size: int = 100,
    deadline: float | None = None,
) -> list[WorkResult[R]]:
    """Run worker pool to process items.
    
    Args:
        items: Work items to process
        processor: Async function to process each item's data
        num_workers: Number of concurrent workers
        queue_size: Maximum queue size (for backpressure)
        deadline: Absolute deadline (from current_time()), None = unlimited
    
    Returns:
        List of work results from all workers
    """
    queue = Queue.with_maxsize(queue_size)
    results = []
    
    async def producer():
        """Feed items into queue."""
        for item in items:
            await queue.put(item)
        # Signal workers to stop
        await queue.put(SENTINEL)
    
    async def run_workers():
        """Spawn and run all workers."""
        async with asyncio.TaskGroup() as tg:
            # Spawn workers
            for i in range(num_workers):
                tg.create_task(worker(
                    worker_id=i,
                    queue=queue,
                    processor=processor,
                    results=results,
                ))
    
    # Run with deadline if specified
    if deadline is not None:
        with move_on_at(deadline):
            async with asyncio.TaskGroup() as tg:
                tg.create_task(producer())
                tg.create_task(run_workers())
    else:
        async with asyncio.TaskGroup() as tg:
            tg.create_task(producer())
            tg.create_task(run_workers())
    
    return results


# Test worker pool with 20 items, 4 workers
async def test_processor(data: dict) -> str:
    await sleep(0.05)  # Simulate work
    return f"Result for user {data['user_id']}"


test_items = [WorkItem(id=f"item-{i}", data={"user_id": i}) for i in range(20)]

start = current_time()
results = await run_worker_pool(
    items=test_items,
    processor=test_processor,
    num_workers=4,
    queue_size=10,
)
elapsed = current_time() - start

print(f"Processed {len(results)} items in {elapsed:.2f}s with 4 workers")
print(f"Expected sequential time: ~{20 * 0.05:.2f}s")
print(f"Speedup: {(20 * 0.05) / elapsed:.1f}x")

# Show per-worker distribution
from collections import Counter
worker_counts = Counter(r.worker_id for r in results)
print(f"\nWork distribution:")
for worker_id, count in sorted(worker_counts.items()):
    print(f"  Worker {worker_id}: {count} items")

**Notes**:
- **Backpressure**: `queue_size=100` limits how many items can be queued. If the producer is faster than workers, it blocks at `queue.put()` until workers catch up.
- **Structured Concurrency**: `TaskGroup` ensures all workers complete (or are cancelled) before returning. No orphaned tasks.
- **Speedup**: With 4 workers and 0.05s tasks, we should see ~4x speedup. Actual speedup depends on queue overhead and task variance.
- **Load Balancing**: Workers pull from a shared queue, so work is automatically balanced - faster workers process more items.

### Step 4: Add Comprehensive Statistics and Monitoring

Production worker pools need detailed metrics: per-worker performance, success/failure rates, queue utilization, deadline compliance. This enables capacity planning and performance tuning.

**Why Per-Worker Stats**: Identifies slow workers (resource contention, bad assignment), enables horizontal scaling decisions.

In [ ]:
@dataclass
class WorkerPoolStats:
    """Statistics from worker pool execution."""
    
    total_items: int
    successful: int
    failed: int
    timeout: int
    total_duration: float
    avg_item_duration: float
    worker_counts: dict[int, int]  # worker_id -> item count
    worker_avg_duration: dict[int, float]  # worker_id -> avg duration
    deadline_reached: bool
    throughput: float  # items per second
    
    def __repr__(self) -> str:
        return (
            f"WorkerPoolStats(\n"
            f"  total={self.total_items}, success={self.successful}, "
            f"failed={self.failed}, timeout={self.timeout}\n"
            f"  duration={self.total_duration:.3f}s, "
            f"avg={self.avg_item_duration:.3f}s, "
            f"throughput={self.throughput:.1f} items/s\n"
            f"  workers={len(self.worker_counts)}, "
            f"deadline_hit={self.deadline_reached}\n"
            f")"
    )


def calculate_stats(
    results: list[WorkResult],
    total_items: int,
    total_duration: float,
    deadline_reached: bool,
) -> WorkerPoolStats:
    """Calculate comprehensive statistics from results."""
    successful = sum(1 for r in results if r.status == WorkStatus.SUCCESS)
    failed = sum(1 for r in results if r.status == WorkStatus.FAILED)
    timeout = sum(1 for r in results if r.status == WorkStatus.TIMEOUT)
    
    # Average duration (only for completed items)
    completed = [r for r in results if r.status in (WorkStatus.SUCCESS, WorkStatus.FAILED)]
    avg_duration = sum(r.duration for r in completed) / len(completed) if completed else 0.0
    
    # Per-worker stats
    worker_counts = {}
    worker_durations = {}
    
    for result in results:
        wid = result.worker_id
        worker_counts[wid] = worker_counts.get(wid, 0) + 1
        
        if result.status in (WorkStatus.SUCCESS, WorkStatus.FAILED):
            if wid not in worker_durations:
                worker_durations[wid] = []
            worker_durations[wid].append(result.duration)
    
    worker_avg_duration = {
        wid: sum(durations) / len(durations)
        for wid, durations in worker_durations.items()
    }
    
    throughput = len(results) / total_duration if total_duration > 0 else 0.0
    
    return WorkerPoolStats(
        total_items=total_items,
        successful=successful,
        failed=failed,
        timeout=timeout,
        total_duration=total_duration,
        avg_item_duration=avg_duration,
        worker_counts=worker_counts,
        worker_avg_duration=worker_avg_duration,
        deadline_reached=deadline_reached,
        throughput=throughput,
    )


async def run_worker_pool_with_stats(
    items: list[WorkItem[T]],
    processor: Callable[[T], R],
    num_workers: int = 4,
    queue_size: int = 100,
    deadline: float | None = None,
) -> tuple[list[WorkResult[R]], WorkerPoolStats]:
    """Run worker pool with comprehensive statistics."""
    start_time = current_time()
    total_items = len(items)
    
    results = await run_worker_pool(
        items=items,
        processor=processor,
        num_workers=num_workers,
        queue_size=queue_size,
        deadline=deadline,
    )
    
    total_duration = current_time() - start_time
    deadline_reached = deadline is not None and current_time() >= deadline
    
    stats = calculate_stats(results, total_items, total_duration, deadline_reached)
    return results, stats


# Test with statistics
test_items = [WorkItem(id=f"item-{i}", data={"user_id": i}) for i in range(50)]

results, stats = await run_worker_pool_with_stats(
    items=test_items,
    processor=test_processor,
    num_workers=5,
    queue_size=20,
)

print(stats)
print("\nPer-worker performance:")
for worker_id in sorted(stats.worker_counts.keys()):
    count = stats.worker_counts[worker_id]
    avg_dur = stats.worker_avg_duration.get(worker_id, 0.0)
    print(f"  Worker {worker_id}: {count} items, avg {avg_dur:.3f}s")

**Notes**:
- **Throughput Metric**: Items per second is the key capacity planning metric. If throughput < required rate, add more workers.
- **Worker Balance**: Ideally, all workers process roughly the same number of items. Large variance indicates resource contention or uneven task complexity.
- **Timeout Tracking**: Separate timeout from failures. Timeouts indicate deadline is too tight; failures indicate data quality or infrastructure issues.
- **Average Duration**: Only includes completed tasks (success + failed). Timeout tasks have zero duration and shouldn't skew the average.

### Step 5: Add Graceful Shutdown and Error Recovery

Production systems need clean shutdown: finish in-flight tasks, flush results, close resources. We also need configurable error handling: retry, circuit breaker, dead letter queue.

**Why Graceful Shutdown**: Abrupt cancellation can lose data (partial writes, uncommitted transactions). Clean shutdown ensures no work is lost.

In [ ]:
@dataclass
class WorkerPoolConfig:
    """Configuration for worker pool."""
    
    num_workers: int = 4
    queue_size: int = 100
    min_time_remaining: float = 0.1  # Skip task if < this time left
    retry_failed: bool = False
    max_retries: int = 2
    on_error: Callable[[WorkItem, Exception], None] | None = None
    on_timeout: Callable[[WorkItem], None] | None = None


async def worker_with_config(
    worker_id: int,
    queue: Queue[WorkItem[T] | _Sentinel],
    processor: Callable[[T], R],
    results: list[WorkResult[R]],
    config: WorkerPoolConfig,
    retry_queue: Queue[WorkItem[T]] | None = None,
) -> None:
    """Worker with full error handling and retry support."""
    while True:
        item = await queue.get()
        
        if item is SENTINEL:
            await queue.put(SENTINEL)
            break
        
        # Check remaining time
        deadline = effective_deadline()
        if deadline is not None:
            remaining = deadline - current_time()
            if remaining < config.min_time_remaining:
                results.append(WorkResult(
                    item_id=item.id,
                    worker_id=worker_id,
                    status=WorkStatus.TIMEOUT,
                ))
                if config.on_timeout:
                    try:
                        config.on_timeout(item)
                    except Exception:
                        pass
                continue
        
        # Process item
        start = current_time()
        try:
            result = await processor(item.data)
            duration = current_time() - start
            results.append(WorkResult(
                item_id=item.id,
                worker_id=worker_id,
                status=WorkStatus.SUCCESS,
                result=result,
                duration=duration,
            ))
        except Exception as e:
            duration = current_time() - start
            
            # Error callback
            if config.on_error:
                try:
                    config.on_error(item, e)
                except Exception:
                    pass
            
            # Retry logic
            retry_count = item.metadata.get("retry_count", 0)
            if config.retry_failed and retry_count < config.max_retries and retry_queue:
                item.metadata["retry_count"] = retry_count + 1
                item.metadata["last_error"] = str(e)
                await retry_queue.put(item)
            else:
                results.append(WorkResult(
                    item_id=item.id,
                    worker_id=worker_id,
                    status=WorkStatus.FAILED,
                    error=e,
                    duration=duration,
                ))


async def run_worker_pool_production(
    items: list[WorkItem[T]],
    processor: Callable[[T], R],
    config: WorkerPoolConfig,
    deadline: float | None = None,
) -> tuple[list[WorkResult[R]], WorkerPoolStats]:
    """Production-ready worker pool with full error handling."""
    start_time = current_time()
    total_items = len(items)
    
    work_queue = Queue.with_maxsize(config.queue_size)
    retry_queue = Queue.with_maxsize(config.queue_size) if config.retry_failed else None
    results = []
    
    async def producer():
        """Feed items into queue."""
        for item in items:
            await work_queue.put(item)
        
        # Process retries if enabled
        if retry_queue:
            while True:
                try:
                    retry_item = retry_queue.get_nowait()
                    await work_queue.put(retry_item)
                except Exception:
                    break
        
        await work_queue.put(SENTINEL)
    
    async def run_workers():
        """Spawn and run all workers."""
        async with asyncio.TaskGroup() as tg:
            for i in range(config.num_workers):
                tg.create_task(worker_with_config(
                    worker_id=i,
                    queue=work_queue,
                    processor=processor,
                    results=results,
                    config=config,
                    retry_queue=retry_queue,
                ))
    
    # Run with deadline if specified
    if deadline is not None:
        with move_on_at(deadline):
            async with asyncio.TaskGroup() as tg:
                tg.create_task(producer())
                tg.create_task(run_workers())
    else:
        async with asyncio.TaskGroup() as tg:
            tg.create_task(producer())
            tg.create_task(run_workers())
    
    total_duration = current_time() - start_time
    deadline_reached = deadline is not None and current_time() >= deadline
    
    stats = calculate_stats(results, total_items, total_duration, deadline_reached)
    return results, stats


# Test with errors and deadline
async def flaky_processor(data: dict) -> str:
    """Processor that fails on some items."""
    await sleep(0.05)
    if data["user_id"] % 7 == 0:
        raise ValueError(f"Validation error for user {data['user_id']}")
    return f"Processed user {data['user_id']}"


def log_error(item: WorkItem, error: Exception):
    print(f"Error processing {item.id}: {error}")


test_items = [WorkItem(id=f"item-{i}", data={"user_id": i}) for i in range(30)]

config = WorkerPoolConfig(
    num_workers=4,
    queue_size=15,
    retry_failed=True,
    max_retries=1,
    on_error=log_error,
)

deadline = current_time() + 2.0  # 2 second deadline
results, stats = await run_worker_pool_production(
    items=test_items,
    processor=flaky_processor,
    config=config,
    deadline=deadline,
)

print("\nFinal Statistics:")
print(stats)

**Notes**:
- **Retry Queue**: Failed items are put in a separate retry queue, preventing retry loops from blocking new work.
- **Callbacks Don't Throw**: Error/timeout callbacks are wrapped in try-except. Callback failures must not break the worker.
- **Graceful Shutdown**: Workers finish their current task before exiting. Sentinel propagation ensures all workers stop.
- **Retry Metadata**: Track retry count and last error in item metadata for debugging and analysis.

## Complete Working Example

Here's the full production-ready implementation combining all steps. Copy-paste this into your project.

**Features**:
- ✅ Worker pool with configurable concurrency
- ✅ Queue-based work distribution with backpressure
- ✅ Sentinel pattern for graceful shutdown
- ✅ Deadline awareness with `effective_deadline()`
- ✅ Comprehensive statistics and monitoring
- ✅ Error handling with retry support
- ✅ Configurable callbacks for observability

In [ ]:
"""Complete production-ready worker pool implementation.

Copy this entire cell into your project and adjust configuration.
"""

# Standard library
import asyncio
from dataclasses import dataclass, field
from enum import Enum
from typing import Any, Callable, Generic, TypeVar

# lionherd-core
from lionherd_core.libs.concurrency import (
    Queue,
    current_time,
    effective_deadline,
    move_on_at,
    sleep,
)

T = TypeVar("T")
R = TypeVar("R")


class _Sentinel:
    """Sentinel marker for graceful shutdown."""
    def __repr__(self) -> str:
        return "SENTINEL"

SENTINEL = _Sentinel()


@dataclass
class WorkItem(Generic[T]):
    """Item to process."""
    id: str
    data: T
    priority: int = 0
    metadata: dict[str, Any] = field(default_factory=dict)


class WorkStatus(Enum):
    SUCCESS = "success"
    FAILED = "failed"
    TIMEOUT = "timeout"


@dataclass
class WorkResult(Generic[R]):
    """Processing result."""
    item_id: str
    worker_id: int
    status: WorkStatus
    result: R | None = None
    error: Exception | None = None
    duration: float = 0.0


@dataclass
class WorkerPoolStats:
    """Pool statistics."""
    total_items: int
    successful: int
    failed: int
    timeout: int
    total_duration: float
    avg_item_duration: float
    worker_counts: dict[int, int]
    worker_avg_duration: dict[int, float]
    deadline_reached: bool
    throughput: float


@dataclass
class WorkerPoolConfig:
    """Pool configuration."""
    num_workers: int = 4
    queue_size: int = 100
    min_time_remaining: float = 0.1
    retry_failed: bool = False
    max_retries: int = 2
    on_error: Callable[[WorkItem, Exception], None] | None = None
    on_timeout: Callable[[WorkItem], None] | None = None


async def worker(
    worker_id: int,
    queue: Queue[WorkItem[T] | _Sentinel],
    processor: Callable[[T], R],
    results: list[WorkResult[R]],
    config: WorkerPoolConfig,
    retry_queue: Queue[WorkItem[T]] | None = None,
) -> None:
    """Worker loop."""
    while True:
        item = await queue.get()
        
        if item is SENTINEL:
            await queue.put(SENTINEL)
            break
        
        # Check deadline
        deadline = effective_deadline()
        if deadline is not None:
            remaining = deadline - current_time()
            if remaining < config.min_time_remaining:
                results.append(WorkResult(
                    item_id=item.id,
                    worker_id=worker_id,
                    status=WorkStatus.TIMEOUT,
                ))
                if config.on_timeout:
                    try:
                        config.on_timeout(item)
                    except Exception:
                        pass
                continue
        
        # Process
        start = current_time()
        try:
            result = await processor(item.data)
            duration = current_time() - start
            results.append(WorkResult(
                item_id=item.id,
                worker_id=worker_id,
                status=WorkStatus.SUCCESS,
                result=result,
                duration=duration,
            ))
        except Exception as e:
            duration = current_time() - start
            
            if config.on_error:
                try:
                    config.on_error(item, e)
                except Exception:
                    pass
            
            retry_count = item.metadata.get("retry_count", 0)
            if config.retry_failed and retry_count < config.max_retries and retry_queue:
                item.metadata["retry_count"] = retry_count + 1
                await retry_queue.put(item)
            else:
                results.append(WorkResult(
                    item_id=item.id,
                    worker_id=worker_id,
                    status=WorkStatus.FAILED,
                    error=e,
                    duration=duration,
                ))


async def run_worker_pool(
    items: list[WorkItem[T]],
    processor: Callable[[T], R],
    config: WorkerPoolConfig | None = None,
    deadline: float | None = None,
) -> tuple[list[WorkResult[R]], WorkerPoolStats]:
    """Run worker pool with deadline awareness.
    
    Args:
        items: Work items to process
        processor: Async function to process item data
        config: Pool configuration
        deadline: Absolute deadline (from current_time())
    
    Returns:
        (results, statistics)
    """
    config = config or WorkerPoolConfig()
    start_time = current_time()
    total_items = len(items)
    
    work_queue = Queue.with_maxsize(config.queue_size)
    retry_queue = Queue.with_maxsize(config.queue_size) if config.retry_failed else None
    results = []
    
    async def producer():
        for item in items:
            await work_queue.put(item)
        if retry_queue:
            while True:
                try:
                    retry_item = retry_queue.get_nowait()
                    await work_queue.put(retry_item)
                except Exception:
                    break
        await work_queue.put(SENTINEL)
    
    async def run_workers():
        async with asyncio.TaskGroup() as tg:
            for i in range(config.num_workers):
                tg.create_task(worker(
                    worker_id=i,
                    queue=work_queue,
                    processor=processor,
                    results=results,
                    config=config,
                    retry_queue=retry_queue,
                ))
    
    if deadline is not None:
        with move_on_at(deadline):
            async with asyncio.TaskGroup() as tg:
                tg.create_task(producer())
                tg.create_task(run_workers())
    else:
        async with asyncio.TaskGroup() as tg:
            tg.create_task(producer())
            tg.create_task(run_workers())
    
    # Calculate statistics
    total_duration = current_time() - start_time
    deadline_reached = deadline is not None and current_time() >= deadline
    
    successful = sum(1 for r in results if r.status == WorkStatus.SUCCESS)
    failed = sum(1 for r in results if r.status == WorkStatus.FAILED)
    timeout = sum(1 for r in results if r.status == WorkStatus.TIMEOUT)
    
    completed = [r for r in results if r.status in (WorkStatus.SUCCESS, WorkStatus.FAILED)]
    avg_duration = sum(r.duration for r in completed) / len(completed) if completed else 0.0
    
    worker_counts = {}
    worker_durations = {}
    for result in results:
        wid = result.worker_id
        worker_counts[wid] = worker_counts.get(wid, 0) + 1
        if result.status in (WorkStatus.SUCCESS, WorkStatus.FAILED):
            if wid not in worker_durations:
                worker_durations[wid] = []
            worker_durations[wid].append(result.duration)
    
    worker_avg_duration = {
        wid: sum(durations) / len(durations)
        for wid, durations in worker_durations.items()
    }
    
    throughput = len(results) / total_duration if total_duration > 0 else 0.0
    
    stats = WorkerPoolStats(
        total_items=total_items,
        successful=successful,
        failed=failed,
        timeout=timeout,
        total_duration=total_duration,
        avg_item_duration=avg_duration,
        worker_counts=worker_counts,
        worker_avg_duration=worker_avg_duration,
        deadline_reached=deadline_reached,
        throughput=throughput,
    )
    
    return results, stats


# Example usage
async def main():
    """Example: Process user notifications with worker pool."""
    
    # Simulate notification sending
    async def send_notification(data: dict) -> str:
        await sleep(0.1)  # Simulate API call
        return f"Sent to user {data['user_id']}"
    
    # Create work items
    items = [
        WorkItem(id=f"notify-{i}", data={"user_id": i})
        for i in range(100)
    ]
    
    # Configure pool
    config = WorkerPoolConfig(
        num_workers=10,
        queue_size=50,
        retry_failed=True,
        max_retries=2,
    )
    
    # Run with 5 second deadline
    deadline = current_time() + 5.0
    results, stats = await run_worker_pool(
        items=items,
        processor=send_notification,
        config=config,
        deadline=deadline,
    )
    
    # Report results
    print(f"Processed {stats.successful}/{stats.total_items} notifications")
    print(f"Failed: {stats.failed}, Timeout: {stats.timeout}")
    print(f"Throughput: {stats.throughput:.1f} items/s")
    print(f"Total time: {stats.total_duration:.2f}s")
    print(f"Deadline reached: {stats.deadline_reached}")


# Run the example
await main()

## Common Issues

**Sentinel Lost**:
- Symptom: Sentinel consumed but not re-queued, workers never exit
- Fix: Always re-queue sentinel after consuming: `await queue.put(SENTINEL)`

**Tasks Started Too Late**:
- Symptom: Workers start tasks with insufficient time to complete
- Fix: Check `effective_deadline() - current_time() >= min_time_remaining`

**Queue Backpressure Issues**:
- Symptom: Producer blocks when queue fills, workers idle when empty
- Fix: Set queue_size = 2-5 × num_workers for balanced buffering

For production patterns (retry logic, monitoring, graceful shutdown), see [lionherd-core Production Guide](https://github.com/khive-ai/lionherd-core/docs/production/deadline_workers.md).

## Variation: Priority Worker Pool

**When to Use**: Work items have different priorities (critical vs nice-to-have)

**Pattern**:
```python
from lionherd_core.libs.concurrency import PriorityQueue

async def run_priority_worker_pool(
    items: list[WorkItem[T]],
    processor: Callable[[T], R],
    config: WorkerPoolConfig,
) -> tuple[list[WorkResult[R]], WorkerPoolStats]:
    # Use PriorityQueue instead of regular Queue
    work_queue = PriorityQueue.with_maxsize(config.queue_size)
    
    # Producer puts items with priority
    for item in items:
        # Higher priority = processed first (negate for max-heap)
        await work_queue.put((-item.priority, item))
    
    # Workers extract: priority, item = await work_queue.get()
    # ... rest similar to base implementation
```

**Trade-offs**:
- ✅ High-priority items processed first (better SLA compliance)
- ✅ Good for mixed workloads (user-facing + background)
- ❌ Low-priority items may starve if high-priority keeps arriving
- ❌ Priority queue ~2x overhead vs FIFO queue

For additional variations (Dynamic Scaling, Two-Phase Workers), see [lionherd-core examples](https://github.com/khive-ai/lionherd-core/examples/deadline_worker_variations.py).

## Summary

**What You Accomplished**:
- ✅ Built production-ready worker pool with `Queue.with_maxsize()` and structured concurrency
- ✅ Implemented sentinel pattern for graceful shutdown without abrupt cancellation
- ✅ Added deadline awareness using `effective_deadline()` to stop processing at deadline
- ✅ Created comprehensive statistics tracking per-worker performance and throughput
- ✅ Implemented error handling with retry logic and configurable callbacks

**Key Takeaways**:
1. **Sentinel pattern** enables graceful shutdown - workers finish current task and exit cleanly without cancellation
2. **Bounded queues** provide backpressure - prevent memory exhaustion when producer is faster than workers
3. **`effective_deadline()`** respects ambient cancel scopes - workers check remaining time before each task
4. **Per-worker statistics** identify performance bottlenecks - uneven work distribution indicates resource contention

**When to Use This Pattern**:
- ✅ High-throughput batch processing (ETL, data transformations, API batch operations)
- ✅ Deadline-bound workloads (nightly jobs, report generation, maintenance windows)
- ✅ I/O-bound tasks where parallelism >> sequential (network calls, database queries)
- ❌ CPU-bound tasks on single-core systems (limited by GIL, use multiprocessing instead)
- ❌ Tasks with complex dependencies (use DAG scheduler instead of queue)

## Related Resources

**lionherd-core API Reference**:
- [Primitives](../../docs/api/libs/concurrency/primitives.md) - `Queue`, `CapacityLimiter`, `Semaphore`
- [Cancellation Utilities](../../docs/api/libs/concurrency/cancel.md) - `effective_deadline()`, `fail_at()`, `move_on_at()`
- [Concurrency Patterns](../../docs/api/libs/concurrency/patterns.md) - `bounded_map()`, `gather()`, `CompletionStream`

**Reference Notebooks**:
- [Concurrency Primitives](../references/concurrency_primitives.ipynb) - Queue, Event, Lock patterns
- [Cancellation Patterns](../references/concurrency_cancel.ipynb) - Deadline and timeout management

**Related Tutorials**:
- [Deadline-Aware Task Queue](./deadline_task_queue.ipynb) - Sequential processing with deadlines
- [Parallel Operations with Timeouts](./parallel_timeouts.ipynb) - `bounded_map()` with deadline awareness

**External Resources**:
- [Structured Concurrency](https://vorpus.org/blog/notes-on-structured-concurrency-or-go-statement-considered-harmful/) - Design principles behind worker pools
- [Python asyncio TaskGroup](https://docs.python.org/3/library/asyncio-task.html#task-groups) - Structured concurrency primitives
- [AWS SQS Best Practices](https://docs.aws.amazon.com/AWSSimpleQueueService/latest/SQSDeveloperGuide/sqs-best-practices.html) - Real-world queue-based processing patterns